In [1]:
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:reporting-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
%use dataframe
%use kandy

In [2]:
import ro.jf.funds.user.sdk.UserSdk
import kotlinx.coroutines.runBlocking

val REPORT_DATA_CONFIGURATION_FILE = "../../data/provision/expenses-report-data-configuration.json"

// TODO(Johann) how could this duplicate code be extracted?
val userSdk = UserSdk()
val username = "Johann-13.1"
val user = runBlocking {
    userSdk.findUserByUsername(username)
        ?: userSdk.createUser(username)
}

user

UserTO(id=51c46d61-4fd6-4a20-b54c-03c5e6f122bc, username=Johann-13.1)

In [3]:
import ro.jf.funds.fund.sdk.FundSdk

val fundSdk = FundSdk()
val expenseFund = runBlocking { fundSdk.getFundByName(user.id, "Expenses") }
expenseFund

FundTO(id=3ce10b59-0db6-4c89-a0d5-58ddae7fe1ed, name=Expenses)

In [17]:
import ro.jf.funds.reporting.api.model.*
import kotlinx.serialization.json.Json

fun getReportDataConfiguration(): ReportDataConfigurationTO {
    val dataConfigurationRawJson: String = File(REPORT_DATA_CONFIGURATION_FILE).readText()
    val jsonFormat = Json { ignoreUnknownKeys = true }
    val dataConfiguration = jsonFormat.decodeFromString<ReportDataConfigurationTO>(dataConfigurationRawJson)
    return dataConfiguration
}
val dataConfiguration = getReportDataConfiguration()
dataConfiguration

ReportDataConfigurationTO(currency=Currency(value=RON), filter=RecordFilterTO(labels=[basic, home, shopping_services, transport, fun, gifts, development]), groups=[ReportGroupTO(name=basic, filter=RecordFilterTO(labels=[basic])), ReportGroupTO(name=home, filter=RecordFilterTO(labels=[home])), ReportGroupTO(name=shopping_services, filter=RecordFilterTO(labels=[shopping_services])), ReportGroupTO(name=transport, filter=RecordFilterTO(labels=[transport])), ReportGroupTO(name=fun, filter=RecordFilterTO(labels=[fun])), ReportGroupTO(name=gifts, filter=RecordFilterTO(labels=[gifts])), ReportGroupTO(name=development, filter=RecordFilterTO(labels=[development]))], features=ReportDataFeaturesConfigurationTO(net=NetReportFeatureTO(enabled=true, applyFilter=true), valueReport=GenericReportFeatureTO(enabled=true), groupedNet=GenericReportFeatureTO(enabled=true), groupedBudget=GroupedBudgetReportFeatureTO(enabled=true, distributions=[BudgetDistributionTO(default=true, from=null, groups=[GroupBudget

In [18]:
import ro.jf.funds.reporting.sdk.ReportingSdk
import kotlinx.coroutines.delay
import ro.jf.funds.commons.model.labelsOf
import ro.jf.funds.commons.model.Currency
import java.time.Instant

val reportingSdk = ReportingSdk()
val reportView = runBlocking {
    val reportViewName = "Expense report"
    val existingReportView = reportingSdk.listReportViews(user.id).items.firstOrNull { it.name == reportViewName }
    if (existingReportView != null) {
        return@runBlocking existingReportView
    }
    val request = CreateReportViewTO(reportViewName, expenseFund.id, dataConfiguration)
    var task: ReportViewTaskTO = reportingSdk.createReportView(user.id, request)
    val timeout = Instant.now().plusSeconds(120)
    while (task is ReportViewTaskTO.InProgress && Instant.now().isBefore(timeout)) {
        delay(2000)
        task = reportingSdk.getReportViewTask(user.id, task.taskId)
    }
    if (task is ReportViewTaskTO.Completed) {
        task.report
    } else if (task is ReportViewTaskTO.Failed) {
        throw IllegalStateException("Report view creation failed on task $task")
    } else {
        throw IllegalStateException("Report view creation timed out on task $task")
    }
}
reportView

ReportViewTO(id=a1a502d3-77d4-4b85-b01b-3ac3407c66dd, name=Expense report, fundId=3ce10b59-0db6-4c89-a0d5-58ddae7fe1ed, dataConfiguration=ReportDataConfigurationTO(currency=Currency(value=RON), filter=RecordFilterTO(labels=[basic, home, shopping_services, transport, fun, gifts, development]), groups=null, features=ReportDataFeaturesConfigurationTO(net=NetReportFeatureTO(enabled=true, applyFilter=true), valueReport=GenericReportFeatureTO(enabled=true), groupedNet=GenericReportFeatureTO(enabled=false), groupedBudget=GroupedBudgetReportFeatureTO(enabled=false, distributions=[]))))

In [19]:
import ro.jf.funds.reporting.api.model.*

val granularInterval = GranularDateInterval(
    interval = DateInterval(
        LocalDate.parse("2019-01-01"),
        LocalDate.parse("2020-12-31")
    ),
    granularity = TimeGranularity.MONTHLY
)
val report = runBlocking {
    reportingSdk.getReportViewData(user.id, reportView.id, granularInterval)
}
report

ReportDataTO(viewId=a1a502d3-77d4-4b85-b01b-3ac3407c66dd, granularInterval=GranularDateInterval(interval=DateInterval(from=2019-01-01, to=2020-12-31), granularity=MONTHLY), data=[ReportDataItemTO(timeBucket=DateInterval(from=2019-01-01, to=2019-01-31), net=-4468.51, value=ValueReportTO(start=0.0, end=2481.52, min=0.0, max=0.0), groups=[ReportDataGroupItemTO(group=basic, net=-2305.04), ReportDataGroupItemTO(group=home, net=-500.0), ReportDataGroupItemTO(group=shopping_services, net=-385.57), ReportDataGroupItemTO(group=transport, net=-204.24), ReportDataGroupItemTO(group=fun, net=-1031.16), ReportDataGroupItemTO(group=gifts, net=0.0), ReportDataGroupItemTO(group=development, net=-42.5)]), ReportDataItemTO(timeBucket=DateInterval(from=2019-02-01, to=2019-02-28), net=-4301.32, value=ValueReportTO(start=2481.52, end=5185.08, min=0.0, max=0.0), groups=[ReportDataGroupItemTO(group=basic, net=-2509.31), ReportDataGroupItemTO(group=home, net=-609.16), ReportDataGroupItemTO(group=shopping_servi

In [20]:
import java.math.BigDecimal
import org.jetbrains.kotlinx.kandy.dsl.* // For creating visualizations
import org.jetbrains.letsPlot.scale.scaleYContinuous

data class MonthlyExpense(val month: LocalDate, val net: Double)

val monthlyReportDF = report.data
    .mapNotNull { reportDataItem ->
        reportDataItem.net?.let { amount ->
        MonthlyExpense(reportDataItem.timeBucket.from, amount.negate().toDouble())
            }
    }
    .toDataFrame()

plot(monthlyReportDF) {
    x("month") {

    }
    bars {
        y("net")

        fillColor = Color.ORANGE
    }
    layout {
        title = "Monthly expenses"
        size = 1800 to 1000
    }
    line {
        y.constant(0)
    }
    line {
        y.constant(10000)
    }
    line {
        y.constant(20000)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="7qiyRF"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Monthly expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12],
"net":[4468.51,4301.32,9171.45,6967.76,9043.57,7024.09,11232.58,25034.218092,4537.4484992,6776.5,9787.68,5529.22,4702.18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
},
"ggsize":{
"width":1800.0,
"height":1000.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month",
"y":"net"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"fill":"#fc8452",
"data":{
}
},{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":10000.0,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":20000.0,
"position":"identity",
"geom":"line",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"column":"month",
"type":"datetime"
}]
}
};
 var plotContainer = document.getElementById("7qiyRF");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M104.92193670801498 921.0 L104.92193670801498 921.0 L171.49131939217477 921.0 L231.61850375206996 921.0 L298.187886436237 921.0 L362.6098696789777 921.0 L429.17925236314477 921.0 L493.60123560588545 921.0 L560.1706182900452 921.0 L626.7400009742123 921.0 L691.161984216953 921.0 L757.73136690112 921.0 L822.1533501438607 921.0 L888.7227328280278 921.0 L955.2921155121949 921.0 L1017.5666993135092 921.0 L1084.1360819976762 921.0 L1148.558065240417 921.0 L1215.127447924584 921.0 L1279.5494311673247 921.0 L1346.1188138514917 921.0 L1412.6881965356588 921.0 L1477.1101797783995 921.0 L1543.6795624625593 921.0 L1608.1015457053072 921.0 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(71,71,71)" stroke-opacity="1.0">
 
 
 
 
 
 <path d="M104.92193670801498 570.622427223657 L104.92193670801498 570.622427223657 L171.49131939217477 570.622427223657 L231.61850375206996 570.622427223657 L298.187886436237 570.622427223657 L362.6098696789777 570.622427223657 L429.17925236314477 570.622427223657 L493.60123560588545 570.622427223657 L560.1706182900452 570.622427223657 L626.7400009742123 570.622427223657 L691.161984216953 570.622427223657 L757.73136690112 570.622427223657 L822.1533501438607 570.622427223657 L888.7227328280278 570.622427223657 L955.2921155121949 570.622427223657 L1017.5666993135092 570.622427223657 L1084.1360819976762 570.622427223657 L1148.558065240417 570.622427223657 L1215.127447924584 570.622427223657 L1279

In [22]:
val categories = mapOf(
    "basic" to Color.GREEN,
    "home" to Color.PEACH,
    "shopping_services" to Color.LIGHT_GREEN,
    "transport" to Color.BLUE,
    "fun" to Color.ORANGE,
    "gifts" to Color.RED,
    "development" to Color.PURPLE,
)

fun getGroupedExpensesDataFrame(): DataFrame<*> {
    val categoriesNetExtractor: Map<String, (List<ReportDataGroupItemTO>) -> Double> =
        categories
            .keys
            .map { group ->
                group to { groups: List<ReportDataGroupItemTO> ->
                    groups.first { it.group == group }.net?.negate()?.toDouble() ?: 0.0
                }
            }
            .toMap()

    return report.data
        .map { monthlyData ->
            monthlyData.timeBucket.from to categoriesNetExtractor.mapValues { (_, extractor) -> extractor(monthlyData.groups!!) }
        }
        .let {
            dataFrameOf(
                "month" to it.flatMap { (month, values) -> values.map { month } },
                "value" to it.flatMap { (_, values) -> values.map { it.value } },
                "category" to it.flatMap { (_, values) -> values.keys }
            )
        }
}
getGroupedExpensesDataFrame().groupBy("category").plot {
    x("month")
    line {
        y("value")
        color("category") {
            scale = categorical(
                *categories.map { (category, color) -> category to color }.toTypedArray()
            )
        }
    }
    line {
        y.constant(0)
    }
    layout {
        title = "Expenses by category"
        size = 1200 to 600
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Eu6SFE"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Expenses by category"
},
"mapping":{
},
"data":{
"&merged_groups":["basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","development","development","development","development","development","development","development","development","development","development","development","development","development","development","development","development","development","development","development","development","development","development","development","development"],
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,

In [23]:
data class MonthlyExpense(val month: LocalDate, val start: Double, val end: Double)

val df = report.data
    .mapNotNull { item ->
        item.value?.let { valueReport ->
            MonthlyExpense(
                item.timeBucket.from,
                valueReport.start.toDouble(),
                valueReport.end.toDouble()
            )
        }
    }
    .let {
        dataFrameOf(
            "month" to it.map { it.month }.let { it + it },
            "value" to it.map { it.start } + it.map { it.end },
            "type" to List(it.size) { "startValue" } + List(it.size) { "endValue" }
        )
    }

df.groupBy("type").plot {
    x("month")
    line {
        y("value")
        color("type") {
            scale = categorical("startValue" to Color.ORANGE, "endValue" to Color.BLUE)
        }
    }
    line {
        y.constant(0)
    }
    layout {
        title = "Expense fund value"
        size = 1200 to 600
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="LIWYMY"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Expense fund value"
},
"mapping":{
},
"data":{
"&merged_groups":["startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue"],
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12],
"type":["startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue"],
"value":[0.0,2481.52,5185.08,11301.26,11528.2727350348,10643.764995302,11871.901585538,8638.3980216284,-6550.7530315744,-873.7461008179,4052.2818580708,6037.1321957254,12396.0035787372,18628.0469211958,18618.2614668309,18611.2675483166,18608.7046926345,18604.8938477149,18607.2871444815,18610.1798150402,18607.0279992326,18595.3562381633,18594.9920505464,18597.026638801,2481.52,5185.08,11301.26,11526.0160226864,10643.310981436,11868.2374648476,8637.9027559568,-6550.8393119068,-872.2869408013,4051.7634872547,6037.1198616282,12394.3761693426,18628.6707953371,18617.2628379781,18609.8185328505,18607.9181809199,18605.0245814626,18608.522806606,18608.6765951979,18607.6628839652,18595.3715021324,18598.4743028244,18595.8096873483,18597.9499255963]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"values":["#fc8452","#5470c6"],
"limits":["startValue","endValue"]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month",
"y":"value",
"color":"type",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"position":"identity",
"geom":"line",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"column":"month",
"type":"datetime"
}]
}
};
 var plotContainer = document.getElementById("LIWYMY");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
